In [1]:
import autorootcwd
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
from src.datasets.splits import get_ottawa2023_splits
from src.datasets.ottawa2023 import Ottawa2023Dataset
from src.datasets.dataset_wrapper import multiplydataset
from src.datasets.transforms import ToTensor, Normalize, FFT, RandomGain
from src.datasets.dataloaders import get_dataloaders
from src.models.dcnn import Net
from src.trainer.train import train
from src.trainer.test import test
import wandb


In [2]:
# --- Configs ---
SEED = 258
BATCH_SIZE = 32
EPOCHS = 20
FAULT_TYPE = "outer"
SAMPLE_LENGTH = 1.0 #in seconds
REPEATS = 2
LEARNING_RATE = 1e-1
AUGMENTATION = 'time-RG'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# DICT OF AUGMENTATIONS
augmentation = { 'fft-RG': [FFT(), RandomGain(seed=SEED), ToTensor()],
                    'fft': [FFT(), ToTensor()],
                    'normalized-fft' : [Normalize(), FFT(), ToTensor()],
                    'normalized-time': [Normalize(), ToTensor()],
                    'time-RG': [RandomGain(seed=SEED), ToTensor()],
                    'time': [ToTensor()] }          
   


In [4]:
# --- Define transforms ---
train_transform = transforms.Compose(augmentation[AUGMENTATION])

val_test_transform = transforms.Compose(augmentation[AUGMENTATION])

In [5]:
wandb.login()

wandb: Currently logged in as: tth-bitt (tth-bitt-universidade-federal-de-santa-catarina) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
wandb.init(project="ottawa2023-fault-detection", 
           name=f"DCNN08_{FAULT_TYPE}_{AUGMENTATION}_mby{REPEATS}_bs{BATCH_SIZE}_lr{LEARNING_RATE}_epochs{EPOCHS}_seed{SEED}",
           config={
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS,
    "learning_rate": LEARNING_RATE,
    "model": "DCNN08",
    "fault_type": FAULT_TYPE,
    "augmentation": AUGMENTATION,
    'seed': SEED,
    'multiplyby': REPEATS,
})

In [7]:
full_df = pd.read_pickle('/data/bearing_datasets/ottawa/processed/full_dataset.bz2')

In [8]:
#split
train_df, val_df, test_df = get_ottawa2023_splits(full_df, random_state=SEED)

In [9]:
# --- Create datasets ---
train_dataset = Ottawa2023Dataset(train_df, faulty_type=FAULT_TYPE, sample_length=SAMPLE_LENGTH, transform=train_transform, random_sample=True, seed=SEED)
val_dataset   = Ottawa2023Dataset(val_df, faulty_type=FAULT_TYPE, sample_length=SAMPLE_LENGTH, transform=val_test_transform, random_sample=False, seed=SEED)
test_dataset  = Ottawa2023Dataset(test_df, faulty_type=FAULT_TYPE, sample_length=SAMPLE_LENGTH, transform=val_test_transform, random_sample=False, seed=SEED)

In [10]:
# --- Multiply training set ---
train_dataset = multiplydataset(train_dataset, repeats=REPEATS)

In [11]:
# --- Create dataloaders ---
train_loader, val_loader, test_loader = get_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=BATCH_SIZE, num_workers=4)

In [12]:
# --- check device ---
print(f"Using device: {DEVICE}")

Using device: cuda


In [13]:
# --- Model, Loss, Optimizer ---
model = Net("DCNN08", in_channels=1, n_class=1).to(DEVICE)
loss_BCE = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [14]:
for epoch in range(EPOCHS):
    train_loss, auroc = train(dataloader = train_loader, model = model, loss_fn = loss_BCE, optimizer = optimizer, device = DEVICE, epoch=epoch)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {train_loss:.4f}, train_AUROC: {auroc:.4f}")
    test(dataloader = val_loader, model = model, loss_fn = loss_BCE, device = DEVICE, epoch = epoch, val_or_test='val')
    test(dataloader = test_loader, model = model, loss_fn = loss_BCE, device = DEVICE, epoch = epoch, val_or_test='test')
    

Epoch 1/20, Loss: 0.3124, train_AUROC: 0.7369
[val] Epoch 0 - Loss: 2.5229, val_AUROC: 0.5795
[test] Epoch 0 - Loss: 0.9860, test_AUROC: 0.4506
Epoch 2/20, Loss: 0.2716, train_AUROC: 0.7887
[val] Epoch 1 - Loss: 0.8213, val_AUROC: 0.4390
[test] Epoch 1 - Loss: 0.6064, test_AUROC: 0.4206
Epoch 3/20, Loss: 0.2926, train_AUROC: 0.7579
[val] Epoch 2 - Loss: 0.7601, val_AUROC: 0.3625
[test] Epoch 2 - Loss: 0.6027, test_AUROC: 0.4941
Epoch 4/20, Loss: 0.2883, train_AUROC: 0.7336
[val] Epoch 3 - Loss: 0.7647, val_AUROC: 0.3205
[test] Epoch 3 - Loss: 0.6252, test_AUROC: 0.3672
Epoch 5/20, Loss: 0.2741, train_AUROC: 0.7422
[val] Epoch 4 - Loss: 0.9676, val_AUROC: 0.3095
[test] Epoch 4 - Loss: 0.6497, test_AUROC: 0.3575
Epoch 6/20, Loss: 0.2790, train_AUROC: 0.7482
[val] Epoch 5 - Loss: 1.0514, val_AUROC: 0.3005
[test] Epoch 5 - Loss: 0.5940, test_AUROC: 0.4659
Epoch 7/20, Loss: 0.2751, train_AUROC: 0.7640
[val] Epoch 6 - Loss: 1.0843, val_AUROC: 0.3095
[test] Epoch 6 - Loss: 0.6548, test_AUROC: